In [1]:
from SimulationSrc import *
import openai
from dotenv import load_dotenv

load_dotenv('.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [13]:
results = []
# 共告知day天的数据
for day in range(1, 6):
    # 重复模拟20次
    for idx in range(1, 21):
        sl = NewsboySimulator()
        data = sl.simulate_constOrder(day)
        print(f'{day} day {idx} times: {data}')
        DemandLambda = sl.get_DemandLambda()
        OrderLambda = sl.get_OrderLambda()
        # 用三种方式推断最佳订货量
        poissonInfer = PoissonInfer(data)
        poissonresult = poissonInfer.infer()
        
        empricalInfer = EmpiricalInfer(data)
        empricalresult = empricalInfer.infer()

        chatGPTInfer = ChatGPTInfer(data)
        chatGPTresult = chatGPTInfer.infer()

        results.append([day, idx, DemandLambda, OrderLambda, poissonresult, empricalresult, chatGPTresult])
        print('finish', day, idx)
# 保存结果
import csv
with open('./results/result_0224.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['day', 'idx', 'DemandLambda', 'OrderLambda' ,'poisson', 'emprical', 'chatGPT'])
    for row in results:
        writer.writerow(row)



1 day 1 times: [(4, 4)]
0.14326268001496317
6.48200263287381
0.08117601253639695
7.548521245454495
0.05588219882944654
8.218738826465463
0.04230908542490655
8.703626648143578
0.033896531841948616
9.081641338720866
0.028194939276980652
9.390393648941556
0.024087323874934567
9.650749994092248
0.02099355900030027
9.875457726568444
0.018583238183024268
10.072858234953046
0.016654732222245904
10.248699200125587
0.015078205827953516
10.407104052539262
0.01376637477975703
10.551128332368226
0.012658450934262695
10.683097276603483
0.011710831079124247
10.804820164967126
0.010891445830692539
10.917731642230931
0.01017619585565784
11.022987908386174
0.009546629514779187
11.121533999223068
0.008988385915703783
11.214151970678405
0.00849012487886668
11.301496127620707
0.008042775264259332
11.384119254621865
0.007638996563464013
11.462492466346012
0.007272786475019184
11.537020449249301
0.006939190362382658
11.608053318636552
0.006634083069674873
11.675895952535152
0.006354002947060664
11.740815418

c:\Users\Lenovo\Desktop\RA\Tao Yao\my_research\ICL_simulation\SimulationSrc.py:104: RuntimeWarning: invalid value encountered in scalar divide
  numer += (i-Lambda)*tmp[i]/Lambda
c:\Users\Lenovo\Desktop\RA\Tao Yao\my_research\ICL_simulation\SimulationSrc.py:107: RuntimeWarning: invalid value encountered in scalar divide
  return -k * numer/denom + np.sum(x[:len(x) - k])/Lambda - (len(x) - k)


finish 1 6
1 day 7 times: [(2, 2)]
0.13577096404625022
4.3167328120956725
0.07234453527580428
5.295117860154564
0.048542155256266525
5.884325323201465
0.03629043740048404
6.302712452283893
0.028874802115721587
6.625772208158981
0.02392265365203615
6.888230581321872
0.020389783718522625
7.1088620240454725
0.01774689777285792
7.298937926972926
0.015697785156421538
7.465743390601334
0.014064042863136948
7.614252910043589
0.012731946390685797
7.748010845664325
0.011625647072706592
7.869629784133934
0.010692655424716068
7.98108965423458
0.009895514359490326
8.083925987190817
0.009206791109097794
8.179353153442056
0.008605942740390862
8.268347746517277
0.008077286524846312
8.35170661717305
0.007608646598357426
8.430088261713008
0.007190428175219408
8.504042974181864
0.006814969813446985
8.574035228630088
0.006476081072966883
8.64046057253544
0.006168706559003816
8.703658568501613
0.005888677852700922
8.763922842116065
0.005632527661001925
8.821508977948051
0.005397348733642696
8.876640793072

In [14]:
import pandas as pd
from SimulationSrc import *
df = pd.read_csv('./results/result_0224.csv')
# 把ChatGPT列中为'None'的行删除
# df = df[df['chatGPT'] != 'None']
df = df[df['poisson'] != 'nan']
# dropna
df = df.dropna()
# df = df[df['poisson'] !='NAN']
# 遍历每一行
# day,idx,DemandLambda,OrderLambda,poisson,emprical,chatGPT
for index, row in df.iterrows():
    DemandLambda = float(row['DemandLambda'])

    poisson_order = int(row['poisson'])
    emprical_order = int(row['emprical'])
    chatGPT_order = int(row['chatGPT'])

    poisson_cost = get_average_cost(poisson_order, DemandLambda)
    emprical_cost = get_average_cost(emprical_order, DemandLambda)
    chatGPT_cost = get_average_cost(chatGPT_order, DemandLambda)


    # 把三种方法的cost和ChatGPT是否beat了poisson和emprical写入csv
    df.at[index, 'poisson_cost'] = poisson_cost
    df.at[index, 'emprical_cost'] = emprical_cost
    df.at[index, 'ChatGPT_cost'] = chatGPT_cost

    # 判断ChatGPT是否beat了poisson和emprical
    if chatGPT_cost < poisson_cost:
        df.at[index, 'ChatGPT_beat_poisson'] = 1
    if chatGPT_cost == poisson_cost:
        df.at[index, 'ChatGPT_eqaul_poisson'] = 1

    if chatGPT_cost < emprical_cost:
        df.at[index, 'ChatGPT_beat_emprical'] = 1
    if chatGPT_cost == emprical_cost:
        df.at[index, 'ChatGPT_eqaul_emprical'] = 1
        
df.to_csv('./results/result_with_cost_0224.csv', index=False)